In [33]:
import geopandas as gpd
import pandas as pd
import numpy as np
from libpysal.graph import read_parquet
from shapely import unary_union

In [34]:

# region_id = 139196 # prague

# region_id = 69333 



# region_id = 8707
# region_id = 5883 # freiburtg
# region_id = 86873
# region_id = 38679
# region_id = 55763
# region_id = 107131
# region_id = 99886

region_id = 151676 # vilnius

# region_id= 66593

In [35]:

etcs = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/tessellations_chars_{region_id}.parquet')
tess = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/tessellations_chars_{region_id}.parquet')
buildings = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/buildings_chars_{region_id}.parquet')
streets = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/streets_chars_{region_id}.parquet')
nodes = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/nodes_chars_{region_id}.parquet')
primary = pd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/primary_chars_{region_id}.parquet')

In [36]:
model_params = '_75_0_None_None_False'
buildings_dir = '/data/uscuni-ulce/processed_data/buildings/'
graph_dir = '/data/uscuni-ulce/processed_data/neigh_graphs/'
morph_dir = '/data/uscuni-ulce/processed_data/morphotopes/'

In [37]:
# streets.explore()

In [38]:
### add the connected structure characters
buildings  = gpd.read_parquet(buildings_dir + f"buildings_{region_id}.parquet")
buildings_q1 = read_parquet(graph_dir + f"building_graph_{region_id}.parquet")
clusters = pd.read_parquet(f'{morph_dir}tessellation_labels_morphotopes_{region_id}_75_0_None_None_False.pq')

In [39]:
buildings['morph'] = clusters
buildings = buildings[buildings.morph.str.split('_').str[-1] != '-1']

In [40]:
groups = buildings.groupby('morph')

In [41]:
group = groups.get_group('568_351')

In [42]:
%%time
group_graph = buildings_q1.subgraph(group.index)
connected_buildings = group.geometry.groupby(group_graph.component_labels).apply( lambda x: unary_union(x.values))
connected_buildings = connected_buildings.set_crs(epsg=3035).buffer(.1).normalize().make_valid()

CPU times: user 27.4 ms, sys: 1.42 ms, total: 28.9 ms
Wall time: 28.2 ms


In [43]:
connected_buildings

component labels
0      POLYGON ((5296735.999 3618470.222, 5296736 361...
1      POLYGON ((5296555.176 3618297.517, 5296555.177...
2      POLYGON ((5296193.729 3618426.965, 5296193.73 ...
3      POLYGON ((5296648.686 3618590.453, 5296648.687...
4      POLYGON ((5296590.005 3618411.849, 5296590.006...
                             ...                        
129    POLYGON ((5296576.716 3618851.898, 5296576.717...
130    POLYGON ((5296259.069 3618501.512, 5296259.069...
131    POLYGON ((5296335.982 3619252.568, 5296335.982...
132    POLYGON ((5296498.67 3618235.342, 5296498.671 ...
133    POLYGON ((5296557.189 3618323.33, 5296557.189 ...
Length: 134, dtype: geometry